In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from IPython.display import clear_output

In [2]:
pd.options.display.max_colwidth = 100

In [3]:
driver = webdriver.Firefox()
driver.get("https://www.google.com/maps/d/u/0/viewer?mid=1suXJNYuMp2OPrc0jXb-61bjnE2c&ll=-5.838466526567973%2C-35.20321450000001&z=15")

In [4]:
bar_title = driver.find_element_by_class_name("suEOdc")
#WAIT TO LOAD
while("UFRN" not in bar_title.text):
    time.sleep(0.5)

In [5]:
list_buttons = driver.find_elements_by_class_name("HzV7m-pbTTYe-KoToPc-ornU0b")
for button in list_buttons:
    button.click()

In [6]:
%%time
stop_items = driver.find_elements_by_class_name("HzV7m-pbTTYe-ibnC6b-V67aGc")
mine_string = ""
i = 0
for stop_item in stop_items:
    if ("Terminal" in stop_item.text) | ("Parada" in stop_item.text):
        i+=1
        stop_item.click()
        time.sleep(0.15)
        fields = driver.find_elements_by_class_name("qqvbed-p83tee-lTBxed")
        for field in fields:
            if "Linhas" in field.text:
                mine_string+="\n"+field.text
        clear_output(wait=True)
        print(str(i)+" paradas mapeadas")
        back = driver.find_element_by_xpath("//span[@class='HzV7m-tJHJj-LgbsSe-Bz112c qqvbed-a4fUwd-LgbsSe-Bz112c']")
        back.click()
        time.sleep(0.15)

66 paradas mapeadas
CPU times: user 1.07 s, sys: 128 ms, total: 1.2 s
Wall time: 1min


In [7]:
driver.close()

In [8]:
print(mine_string)


Linhas:
> 588 - Circular UFRN (direto)
> 588 - Circular UFRN (Inverso)

> Empresas: Guanabara, Reunidas, Santa Maria, ViaSul e Cidade do Natal
> Linhas de ônibus:
* 18 - Bairro Nordeste/Campus;
* 31 - Felipe Camarão/Pirangi, via Campus/Candelária;
* 48 - Santos Reis/Nova Descoberta, via Alecrim/Campus;
* 57 - Mãe Luíza/Nova Descoberta, via Campus;
* 63 - Felipe Camarão/Campus, via Av. Bernardo Vieira;
* 66 - Ponta Negra/Cidade da Esperança, via Bom Pastor;
* 588 - Circular UFRN (direto)
> Linhas de ônibus:
* 18 - Bairro Nordeste/Campus;
* 31 - Felipe Camarão/Pirangi, via Campus/Candelária;
* 48 - Santos Reis/Nova Descoberta, via Alecrim/Campus;
* 57 - Mãe Luíza/Nova Descoberta, via Campus;
* 63 - Felipe Camarão/Campus, via Av. Bernardo Vieira;
* 66 - Ponta Negra/Cidade da Esperança, via Bom Pastor;
* 588 - Circular UFRN (direto)
> Linhas de ônibus:
* 31 - Felipe Camarão/Pirangi, via Campus/Candelária;
* 48 - Santos Reis/Nova Descoberta, via Alecrim/Campus;
* 57 - Mãe Luíza/Nova Descob

In [9]:
string_list = mine_string.split("\n")
string_list = [x for x in string_list if x!='']
string_list = [x for x in string_list if "Linhas" not in x]
string_list = [x for x in string_list if "Empresas" not in x]
string_list = [x for x in string_list if "> " not in x]
dataframe = pd.DataFrame(list(set(string_list)),columns=["completo"])
dataframe["linha"] = dataframe.completo.str.extract(r"\* ([\w\/]+) -")
dataframe["nome"] = dataframe.completo.str.extract(r"- ([\w\s\d\/\(\).,]+)")
dataframe = dataframe.dropna().loc[:,["linha","nome"]].sort_values(by=["linha","nome"])
display(dataframe.reset_index(drop=True))

,linha,nome
0,10/29,"Nova Natal/Nova Descoberta, via Av. da Chegança/Campus"
1,10/29,"Nova Natal/Nova Descoberta, via Cidade Praia/Campus"
2,18,Bairro Nordeste/Campus
3,30,"Felipe Camarão/Pirangi, via Candelária/Campus"
4,31,"Felipe Camarão/Pirangi, via Campus/Candelária"
5,48,"Santos Reis/Nova Descoberta, via Alecrim/Campus"
6,54A,"Ponta Negra/Campus, via Rua Walter Duarte Pereira"
7,57,"Mãe Luíza/Nova Descoberta, via Campus"
8,588,Circular UFRN (Inverso)
9,588,Circular UFRN (direto)


In [10]:
dataframe.to_csv("linhas_diretas_ufrn.csv",index=False)